# TITLE

# Introduction

To fill

<div style="border-left: 6px solid rgba(69, 157, 185, 1);border-radius:5px; box-shadow: 3px 3px 3px rgba(221, 221, 221, 1);" >
    <p style="background-color: rgba(69, 157, 185, 0.1); font-weight:bold; padding: 8px 0 8px 15px;">Analysis</p>
    <div style="padding: 0 0 2px 10px;">
    
**What will be covered :**
- **Part 1 :** fill
- **Part 2 :** fill
- **Part 3 :** fill

</div></div>
<br/>

---

# Part 1: Movie Dataset

### Import libraries

In [115]:
import numpy as np
import pandas as pd

# LDA
# gensim is a popular library for topic modelling
# pip install gensim
# pip install nltk
# pip install kagglehub
# pip install spacy
# pip install IMDbPY
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from imdb import IMDb
import nltk
import re
import kagglehub # To extract synopsis dataframes
import spacy


nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /Users/serge/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/serge/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Load data

In [106]:
movie_columnns =['Wikipedia movie ID','Freebase movie ID', 'Movie name', 'Movie release date', 'Movie box office revenue', 'Movie runtime', 'Movie languages', 'Movie countries', 'Movie genres']
movie = pd.read_csv('Data/movie.metadata.tsv', sep='\t', header=None, names=movie_columnns)
movie.head()

,Wikipedia movie ID,Freebase movie ID,Movie name,Movie release date,Movie box office revenue,Movie runtime,Movie languages,Movie countries,Movie genres
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp..."
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D..."
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic..."
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}"


## Latent Dirichlet Allocation (LDA)
To group movies together based on something more precise than their genre, we will use Latent Dirichlet Allocation. LDA offers a way to cluster together movies that reflect similar themes, based on textual descriptions. The model is based on unsupervised learning, as the subjects are not known. However the movie summaries only give us too few informations to use LDA. That's why we will extract the synopsis of the movies through another dataset called "Movie Plot Synopses with Tags" (MPST). Those synopsis are around 10 times longer than the summaries, giving way more informations to the LDA model.

### Exploratory Data Analysis of Synopsis Dataframe

In [107]:
# Load data
synopsis = pd.read_csv('Data/mpst_full_data.csv')

# Check data shape
print("This synopsis dataframe is of size:", synopsis.shape)

# Display df
synopsis.head()

This synopsis dataframe is of size: (14828, 6)


,imdb_id,title,plot_synopsis,tags,split,synopsis_source
0,tt0057603,I tre volti della paura,Note: this synopsis is for the orginal Italian...,"cult, horror, gothic, murder, atmospheric",train,imdb
1,tt1733125,Dungeons & Dragons: The Book of Vile Darkness,"Two thousand years ago, Nhagruul the Foul, a s...",violence,train,imdb
2,tt0033045,The Shop Around the Corner,"Matuschek's, a gift store in Budapest, is the ...",romantic,test,imdb
3,tt0113862,Mr. Holland's Opus,"Glenn Holland, not a morning person by anyone'...","inspiring, romantic, stupid, feel-good",train,imdb
4,tt0086250,Scarface,"In May 1980, a Cuban man named Tony Montana (A...","cruelty, murder, dramatic, cult, violence, atm...",val,imdb


### Comment
The dataframe offers extra movie tags and the full movie synopsis which would give the LDA more context to group movies together. On the other hand there a only 14'828 rows/movies which is consequently smaller than the summaries dataframe. We make the choice to reduce the dataset but gain precision on the themes generated by the LDA. Another important notice is that this dataframe uses imdb_id not wikipedia movie ids.

### Adding synopsis and tags to the initial movie df
To match the two dataset, we will merge by the name of the movies

In [108]:
movies_synopsis = synopsis.merge(movie, left_on='title', right_on='Movie name', how='inner')
movies_synopsis.drop(columns=['title', 'split'], inplace=True)
movies_synopsis.head()

,imdb_id,plot_synopsis,tags,synopsis_source,Wikipedia movie ID,Freebase movie ID,Movie name,Movie release date,Movie box office revenue,Movie runtime,Movie languages,Movie countries,Movie genres
0,tt1733125,"Two thousand years ago, Nhagruul the Foul, a s...",violence,imdb,30855958,/m/0gfjl1f,Dungeons & Dragons: The Book of Vile Darkness,2012-08-09,NaN,90.0,{},"{""/m/07ssc"": ""United Kingdom""}","{""/m/01hmnh"": ""Fantasy""}"
1,tt0033045,"Matuschek's, a gift store in Budapest, is the ...",romantic,imdb,76353,/m/0k4bt,The Shop Around the Corner,1940-01-12,NaN,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06cvj"": ""Romantic comedy"", ""/m/0hj3nyp"": ..."
2,tt0113862,"Glenn Holland, not a morning person by anyone'...","inspiring, romantic, stupid, feel-good",imdb,171076,/m/016z98,Mr. Holland's Opus,1995-12-29,106269971.0,130.0,"{""/m/02h40lc"": ""English Language"", ""/m/0my5"": ...","{""/m/09c7w0"": ""United States of America""}","{""/m/0hj3n84"": ""Inspirational Drama"", ""/m/0hqx..."
3,tt0086250,"In May 1980, a Cuban man named Tony Montana (A...","cruelty, murder, dramatic, cult, violence, atm...",imdb,76331,/m/0k44g,Scarface,1932,NaN,94.0,"{""/m/02bjrlw"": ""Italian Language"", ""/m/02h40lc...","{""/m/09c7w0"": ""United States of America""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/0gw5w78"": ""G..."
4,tt0086250,"In May 1980, a Cuban man named Tony Montana (A...","cruelty, murder, dramatic, cult, violence, atm...",imdb,267848,/m/01nln3,Scarface,1983-12-01,65884703.0,170.0,"{""/m/02h40lc"": ""English Language"", ""/m/06nm1"":...","{""/m/09c7w0"": ""United States of America""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/01jfsb"": ""Th..."


In [109]:
lost_rows = synopsis.shape[0] - movies_synopsis.shape[0]
print(f"We only lost {lost_rows} movies by merging the two dataset by the titles of the movies.")

We only lost 94 movies by merging the two dataset by the titles of the movies.


### Comment

1. Movie loss


The size fo the dataframe has not significantly changed, only 94 movies did not match. Our original movie df contained most of the movies present in the synopsis df.

2. Same title


We also observe that Scarface appears twice since there was a remake. For the rest for the analysis we will assume that if the name of the movie is exactly the same, the synopsis is pretty similar for both movies (if not the same!).

# Can we observe new movies' groups using LDA?

## Preprocessing
Removing stop words and names before using LDA is important as those very frequent words are not used to predict themes + lowercase 

In [110]:
# This cell takes long to run you can use the movies_synopsis df instead

"""
# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

def remove_named_entities(text):
    doc = nlp(text)
    return ' '.join([token.text for token in doc if token.ent_type_ != "PERSON"])  # Exclude PERSON entities

stop_words = set(stopwords.words('english'))

# Sample movie summaries
sentences = movies_synopsis['plot_synopsis'].tolist()
print("Their are", len(sentences), "sentences")

# Remove names
cleaned_sentences= [remove_named_entities(doc) for doc in sentences]
print("After removing names their are", len(cleaned_sentences), "sentences")

# Remove stop words lowercase
processed_sentences = [
    [word for word in word_tokenize(sentence.lower()) if word.isalnum() and word not in stop_words]
    for sentence in cleaned_sentences
]
print("After total processing their are", len(processed_sentences), "sentences")

# stores processed_sentences in a df so that those steps do not have to be repeated
movies_synopsis['processed synopsis'] = processed_sentences
movies_synopsis.to_csv('movies_synopsis.csv', index=False)
"""


'\n# Load the spaCy model\nnlp = spacy.load("en_core_web_sm")\n\ndef remove_named_entities(text):\n    doc = nlp(text)\n    return \' \'.join([token.text for token in doc if token.ent_type_ != "PERSON"])  # Exclude PERSON entities\n\nstop_words = set(stopwords.words(\'english\'))\n\n# Sample movie summaries\nsentences = movies_synopsis[\'plot_synopsis\'].tolist()\nprint("Their are", len(sentences), "sentences")\n\n# Remove names\ncleaned_sentences= [remove_named_entities(doc) for doc in sentences]\nprint("After removing names their are", len(cleaned_sentences), "sentences")\n\n# Remove stop words lowercase\nprocessed_sentences = [\n    [word for word in word_tokenize(sentence.lower()) if word.isalnum() and word not in stop_words]\n    for sentence in cleaned_sentences\n]\nprint("After total processing their are", len(processed_sentences), "sentences")\n\n# stores processed_sentences in a df so that those steps do not have to be repeated\nmovies_synopsis[\'processed synopsis\'] = proces

In [111]:
# Create a dictionary and corpus for the LDA model
movies_synopsis = pd.read_csv('Data/movies_synopsis.csv')
processed_sentences = movies_synopsis['processed synopsis'].tolist()

FileNotFoundError: [Errno 2] No such file or directory: 'Data/movies_synopsis.csv'

In [ ]:
dictionary = Dictionary(processed_sentences)
corpus = [dictionary.doc2bow(sentence) for sentence in processed_sentences]

# Train LDA model
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=10, random_state=42, passes=10, iterations=50)

TypeError: doc2bow expects an array of unicode tokens on input, not a single string

In [ ]:
for topic_id, topic_words in lda_model.print_topics(num_words=15):
    print(f"Topic {topic_id}: {topic_words}")

Topic 0: 0.016*"police" + 0.008*"car" + 0.007*"man" + 0.007*"kill" + 0.007*"killed" + 0.007*"house" + 0.005*"finds" + 0.005*"dead" + 0.005*"gun" + 0.005*"one" + 0.005*"find" + 0.005*"body" + 0.005*"death" + 0.005*"murder" + 0.005*"shoots"
Topic 1: 0.005*"one" + 0.004*"war" + 0.004*"team" + 0.004*"men" + 0.003*"ship" + 0.003*"escape" + 0.003*"killed" + 0.003*"two" + 0.003*"attack" + 0.003*"world" + 0.003*"army" + 0.003*"earth" + 0.003*"group" + 0.003*"bomb" + 0.003*"time"
Topic 2: 0.022*"hamlet" + 0.018*"king" + 0.012*"macbeth" + 0.009*"claudius" + 0.006*"prince" + 0.006*"banquo" + 0.005*"act" + 0.005*"witches" + 0.005*"ghost" + 0.005*"queen" + 0.004*"castle" + 0.004*"musketeers" + 0.004*"father" + 0.004*"england" + 0.004*"love"
Topic 3: 0.007*"father" + 0.005*"love" + 0.005*"family" + 0.005*"one" + 0.005*"life" + 0.005*"new" + 0.005*"mother" + 0.004*"film" + 0.004*"home" + 0.004*"time" + 0.004*"also" + 0.004*"wife" + 0.003*"son" + 0.003*"later" + 0.003*"two"
Topic 4: 0.007*"back" + 0.0

This approcah seems promising! With very basic preprocessing can already interpret some themes:

*   Topic 0: Police + Kill -> Detective
*   Topic 1: War + team --> War
*   Topic 2: King + witches --> Medevial
*   Topic 3: Father + Love --> Family
*   Topic 4: Twon + Sherrif + Horse --> Western
*   Topic 5: Sea + treasure --> Pirate
*   Topic 6: Prince + Castle --> Fairy tale
*   Topic 7: Money + job --> Buisness
*   Topic 8: ?
*   Topic 9: ?


But we also observe that this still has room for improvement in example topics 9 still carries many names thus for further analysis the name removal step should be fine tuned. And also lemmatization should be applied to avoid reoccurences of declinaison of the same word (e.g killed and kill in topic 0)







passes: refers to the number of complete passes through the entire dataset. Increasing it gives the model more chances to learn the structure of your data.

iterations controls how many times the model iterates through each document per pass. Increasing iterations can help improve convergence for smaller datasets.

### Retrieving NaN values in box-office data
After realizing the high number of NaN values in the box-office revenue data, we decided to complete as much as possible this data with the IMDB library.

In [131]:
def clean_ids(s, pattern=' '):
    s = str(s).replace(pattern, '')
    return s

#Get the right imdb_id by removing 'tt' in front of IDs
movies_synopsis['imdb_id'] = movies_synopsis['imdb_id'].apply(lambda x : clean_ids(x, 'tt'))

#Create a dic telling if 'Movie box office revenue' is a NaN or not
isnan_dic = movies_synopsis['Movie box office revenue'].isna()

def get_box_office(imdb_ID): #return the box-office revenue for a specific imdb_id
    ia = IMDb()
    '''try:
        movie = ia.get_movie(imdb_ID)
        # Attempt to get the specific 'Cumulative Worldwide Gross' box office info
        box_office = movie.get('box office', {}).get('Cumulative Worldwide Gross', np.nan)
        box_office_num = float(re.sub(r'[^\d.]', '', box_office))
    except Exception as e:
        print(f"Error retrieving box office for {imdb_ID}: {e}")
        box_office_num = np.nan
    return box_office_num'''
    movie = ia.get_movie(imdb_ID)
    box_office = movie.get('box office')
    if box_office and 'Cumulative Worldwide Gross' in box_office:
        box_office_num = float(re.sub(r'[^\d.]', '', box_office['Cumulative Worldwide Gross']))
        #box_office_num = float(box_office['Cumulative Worldwide Gross'].replace('$', '').replace(',', ''))
        return box_office_num
    return None  # Return None if no box office data is found


<>:13: SyntaxWarning: invalid escape sequence '\d'
<>:13: SyntaxWarning: invalid escape sequence '\d'
/var/folders/cp/qf3c3lts5zgc4q5ppr1pc6qw0000gn/T/ipykernel_89250/1995892226.py:13: SyntaxWarning: invalid escape sequence '\d'
  '''try:


In [132]:
#Quick checks !!! Change indices if new dataset
index_case1 = 0
index_case2 = 1
index_case3 = 2
print(f"The initial box-office was {movies_synopsis.iloc[index_case1]['Movie box office revenue']}, we find with Imdb library {get_box_office(movies_synopsis.iloc[index_case1]['imdb_id'])}")
print(f"The initial box-office was {movies_synopsis.iloc[index_case2]['Movie box office revenue']}, we find with Imdb library {get_box_office(movies_synopsis.iloc[index_case2]['imdb_id'])}")
print(f"The initial box-office was {movies_synopsis.iloc[index_case3]['Movie box office revenue']}, we find with Imdb library {get_box_office(movies_synopsis.iloc[index_case3]['imdb_id'])}")

The initial box-office was nan, we find with Imdb library None
The initial box-office was nan, we find with Imdb library 380000.0
The initial box-office was 106269971.0, we find with Imdb library 106269971.0


In [ ]:
'''movies_synopsis.to_csv('test_thomas.csv', index=False)''' #Just testing something

We are indeed getting more box-office data !

In [ ]:
#Apply on the whole dataset !!! TOO LONG
#movies_synopsis['IMDB Box-office'] = movies_synopsis.apply(lambda x: get_box_office(x['imdb_id']) if isnan_dic[x.name] else x['Movie box office revenue'], axis=1)

KeyboardInterrupt: 

In [134]:
new_box = pd.read_csv('with_new_box_office.csv')
new_box

,imdb_id,plot_synopsis,tags,synopsis_source,Wikipedia movie ID,Freebase movie ID,Movie name,Movie release date,Movie box office revenue,Movie runtime,Movie languages,Movie countries,Movie genres,IMDB Box-office
0,1733125,"Two thousand years ago, Nhagruul the Foul, a s...",violence,imdb,30855958,/m/0gfjl1f,Dungeons & Dragons: The Book of Vile Darkness,2012-08-09,NaN,90.0,{},"{""/m/07ssc"": ""United Kingdom""}","{""/m/01hmnh"": ""Fantasy""}",NaN
1,33045,"Matuschek's, a gift store in Budapest, is the ...",romantic,imdb,76353,/m/0k4bt,The Shop Around the Corner,1940-01-12,NaN,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06cvj"": ""Romantic comedy"", ""/m/0hj3nyp"": ...",380000.0
2,113862,"Glenn Holland, not a morning person by anyone'...","inspiring, romantic, stupid, feel-good",imdb,171076,/m/016z98,Mr. Holland's Opus,1995-12-29,106269971.0,130.0,"{""/m/02h40lc"": ""English Language"", ""/m/0my5"": ...","{""/m/09c7w0"": ""United States of America""}","{""/m/0hj3n84"": ""Inspirational Drama"", ""/m/0hqx...",106269971.0
3,86250,"In May 1980, a Cuban man named Tony Montana (A...","cruelty, murder, dramatic, cult, violence, atm...",imdb,76331,/m/0k44g,Scarface,1932,NaN,94.0,"{""/m/02bjrlw"": ""Italian Language"", ""/m/02h40lc...","{""/m/09c7w0"": ""United States of America""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/0gw5w78"": ""G...",65884703.0
4,86250,"In May 1980, a Cuban man named Tony Montana (A...","cruelty, murder, dramatic, cult, violence, atm...",imdb,267848,/m/01nln3,Scarface,1983-12-01,65884703.0,170.0,"{""/m/02h40lc"": ""English Language"", ""/m/06nm1"":...","{""/m/09c7w0"": ""United States of America""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/01jfsb"": ""Th...",65884703.0
